In [ ]:
# Instalando las dependencias ...

!pip install ir-datasets nltk spacy

import nltk
nltk.download('wordnet')

!python -m spacy download en_core_web_sm

# Laboratorio #4: Expasión de la consulta

En los Sistemas de Recuperación de Información la **expansión de consultas** es una técnica utilizada para mejorar la precisión y la exhaustividad de los resultados de búsqueda. Consiste en ampliar la consulta original del usuario agregando términos relevantes que puedan mejorar la recuperación de información.

De las técnicas más utilizadas para expandir consultas en esta clase se verán:uso de sinónimos y matriz de co-ocurrencia.

Luego, el objetivo de este laboratorio es implementar estas dos técnicas.

In [28]:
import nltk
from nltk.corpus import wordnet
from nltk.wsd import lesk

import ir_datasets
dataset = ir_datasets.load("cranfield")

import spacy
nlp = spacy.load("en_core_web_sm")

Antes de comenzar a implementar las técnicas, debe de preparar el escenario. Para ello tiene que procesar el texto e identificar los tokens que lo caracterizó.

Conozca que `Token.pos_` devuelve la etiqueta gramatical del token. Las etiquetas se rigen por las categorías universales siguientes:
  - ADJ: adjetivo
  - ADP: aposición
  - ADV: adverbio
  - AUX: auxiliar
  - CCONJ: conjunción coordinante
  - DET: determinante
  - INTJ: interjección
  - NOUN: sustantivo
  - NUM: número
  - PARTE: partícula
  - PRON: pronombre
  - PROPN: nombre propio
  - SCONJ: conjunción subordinante
  - SYMBOL: símbolo
  - VERB: verbo
  - X: otro

### **Ejercicio #1:** Determine las etiquetas que servirán como filtro para caracterizar a cada documento.

In [43]:
# Not Implemented
tag_list = []

In [44]:
def tokenize(text : str):
  """
  Tokenize and select text tokens

  Args:
    - text (str) : Text.

  Return:
    list<str>

  """
  global tag_list

  doc = nlp(text)
  tokens = []
  for token in doc:
    if token.pos_ in tag_list:
      tokens.append(token.lemma_)

  return tokens


## Técnica #1: Uso de la matriz de co-ocurrencia

La matriz de co-ocurrencia es una representación tabular que muestra la cantidad de veces que ocurre cada par de tokens en un contexto determinado. El contexto puede ser una ventana de palabras alrededor de un token específico o incluso el documento completo, siendo este último el más utilizado en procesamiento de corpus pequeños. Cada celda de la matriz indica el número de veces que dos tokens aparecen juntos en el mismo contexto. Además, esta matriz captura relaciones entre los tokens y es simétrica.

Esta matriz se construye al inicio del sistema y se utiliza posteriormente cada vez que sea necesaria.

### **Ejercicio #2:** Implemente la contrucción de la matriz de co-ocurrencia.

In [ ]:
def build_co_occurrence_matrix(docs):
  """
  Constructs the occurrence matrix from a tokenized corpus. Uses the document as a window.

  Args:
    - docs (list<list<str>>) : Tokenized corpus.

  Return:
    - `look for your best representation`

  """
  return NotImplemented('Resuelva el ejercicio #2')

In [ ]:
docs = [tokenize(doc) for (_, _, doc, _, _) in dataset.docs_iter()]
co_occurrence_matrix = build_co_occurrence_matrix(docs)

Una vez construida la matriz de co-ocurrencia, es posible implementar la técnica de expansión de consultas.

### **Ejercicio #3:** Implemente la expansión de consultas teniendo en cuenta la matriz de co-ocurrencia.

In [46]:
def expand_with_co_occurrence_matrix(query, co_occurrence_matrix):
  """
  Expand the query using co-occurrence matrix

  Args:
    - query (list<str>) : Tokenized query
    - co_occurrence_matrix (??) : Co-occurrence matrix

  Return:
    list<str>

  """

  raise NotImplementedError('Resuelva el ejercicio #3')

In [ ]:
query = "Could you provide insights on the relationship between theory, measurement, and model, and how they converge at a certain point?"
print('Query: ', query)

tokenized_query = tokenize(query)
print('Tokenized query: ', tokenized_query)

tmp = expand_with_co_occurrence_matrix(query)
print('expanded query using co-occurrence matrix: ', tmp)

## Técnica #2: Uso de WordNet

Wordnet es una base de datos léxica para el idioma inglés que funciona como una red semántica. Cada palabra tiene un conjunto de conceptos correspondientes en Wordnet denominados synsets. Un synset representa el significado de una palabra en un contexto específico.

### Relaciones Semánticas entre Synsets

- **Hipónimos**: Los hipónimos son synsets que representan subtipos o instancias específicas de un concepto más general. Por ejemplo, para el synset de "árbol", los hipónimos incluirían tipos específicos de árboles como "roble", "pino", etc. Los hipónimos proporcionan una relación de "es un tipo de".

- **Hiperónimos**: Los hiperónimos son lo opuesto a los hipónimos; son synsets que representan categorías más generales bajo las cuales se clasifican varios synsets más específicos. Siguiendo el ejemplo anterior, "planta" sería un hiperónimo de "árbol". Los hiperónimos proporcionan una relación de "es una categoría de".

- **Merónimos**: Los merónimos describen una relación de parte-todo. Hay varios tipos de merónimos:
    - Parte (Part Meronyms): Indican que el synset es una parte de algo más grande. Por ejemplo, "rueda" es un parte merónimo de "automóvil".
    - Miembro (Member Meronyms): Indican que el synset es un miembro de un conjunto o grupo. Por ejemplo, "árbol" es un miembro merónimo de "bosque".
    - Sustancia (Substance Meronyms): Indican que el synset es una sustancia que compone algo más grande. Por ejemplo, "madera" es un sustancia merónimo de "árbol".

- **Holónimos**: Los holónimos describen la relación inversa a los merónimos; indican que el synset es un todo que incluye las partes, miembros o sustancias representadas por otros synsets. Hay varios tipos:
    - Parte (Part Holonyms): Indican que el synset incluye al synset dado como una parte. Por ejemplo, "automóvil" es un parte holónimo de "rueda".
    - Miembro (Member Holonyms): Indican que el synset incluye al synset dado como un miembro. Por ejemplo, "bosque" es un miembro holónimo de "árbol".
    - Sustancia (Substance Holonyms): Indican que el synset incluye al synset dado como una sustancia. Por ejemplo, "árbol" es un sustancia holónimo de "madera".

- **Antónimos**: Los antónimos son lemas en synsets opuestos o contrastantes en significado. Por ejemplo, "frío" es antónimo de "caliente". La relación de antonimia es directa y se encuentra principalmente a nivel de lemas dentro de los synsets.

- **Entailments** (para verbos): Las implicaciones o entailments describen una relación donde un verbo implica otro. Por ejemplo, "roncar" implica "dormir". Si alguien ronca, entonces necesariamente está durmiendo.

La función `wordnet.synsets(word)` devuelve la lista de synsets asociado a `word`.

### Funciones Principales sobre Synsets

- `definition()`: Devuelve la definición del Synset.
- `examples()`: Muestra ejemplos de uso.
- `lemmas()`: Lista los lemas asociados al Synset.
- `name()`: Retorna el nombre identificador del Synset.
- `lemma_names()`: Obtiene los nombres de los lemas.
- `hypernyms()`: Busca Synsets más generales (conceptos padres).
- `hyponyms()`: Encuentra Synsets más específicos (subconceptos).
- `part_meronyms()`: Identifica las partes constituyentes de un Synset.
- `substance_meronyms()`: Encuentra sustancias que componen el Synset.
- `member_meronyms()`: Lista miembros de un grupo o conjunto que representa el Synset.
- `part_holonyms()`: Synsets de los cuales este Synset es parte.
- `substance_holonyms()`: Synsets que contienen este Synset como sustancia.
- `member_holonyms()`: Grupos o conjuntos a los que este Synset pertenece.
- `antonyms()`: Encuentra opuestos a través de los lemas (debe accederse a través de lemas específicos).
- `entailments()`: (Para verbos) Acciones que implica realizar el verbo.
- `also_sees()`: Synsets conceptualmente relacionados que no se ajustan a otras relaciones.
- `verb_groups()`: Para verbos, otros verbos que están conceptualmente relacionados.
- `attributes()`: Atributos asociados al Synset para adjetivos.
- `similar_tos()`: Synsets que son similares en concepto.
- `topic_domains()`: Devuelve los dominios temáticos asociados al Synset.
- `region_domains()`: Devuelve los dominios regionales asociados al Synset.
- `usage_domains()`: Devuelve los dominios de uso asociados al Synset.
- `causes()`: (Para verbos) Eventos o estados que el verbo causa.

Estas relaciones semánticas entre Synsets facilitan una comprensión de la estructura semántica del léxico, siendo muy útiles para aplicaciones en procesamiento del lenguaje natural.

### **Ejercicio #4:**  Escoja la definición correcta para cada palabra de la consulta.

In [ ]:
def get_synsets_for_context(query):
    """
    Get correct definition for each word in query for the given context

    Args:
        - query (list<str>) : Tokenized query

    Return:
        list<Synset>

    """
    
    raise NotImplementedError('Resuelva el ejercicio #4')

### **Ejercicio #5:**  Implemente la expansión de consultas aprovechando las relaciones semánticas.

In [ ]:
def expand_with_wordnet(query):
  """
  Expand the query using synonyms

  Args:
    - query (list<str>) : Tokenized query

  Return:
    list<str>

  """
  synsets = get_synsets_for_context(query)
  
  raise NotImplementedError('Resuelva el ejercicio #5')

In [ ]:
query = "Could you provide insights on the relationship between theory, measurement, and model, and how they converge at a certain point?"
print('Query: ', query)

tokenized_query = tokenize(query)
print('Tokenized query: ', tokenized_query)

tmp = expand_with_wordnet(query)
print('Expanded query using synonyms: ', tmp)